In [ ]:
import tensorflow as tf
from sklearn.model_selection import train_test_split
import tensorflow_datasets as tfds
from keras.api.layers import Dense, Embedding, Activation
from keras.api import Input
from keras.api.models import Model
from transformers.models.bert import TFBertTokenizer, TFBertEmbeddings  # embedding and tokenizer for description/nlp related stuff

In [ ]:
# !pipreqsnb . --force

pipreqs  --force .


INFO: Not scanning for jupyter notebooks.
<unknown>:165: SyntaxWarning: invalid escape sequence '\S'
<unknown>:166: SyntaxWarning: invalid escape sequence '\['
<unknown>:207: SyntaxWarning: invalid escape sequence '\['
<unknown>:456: SyntaxWarning: invalid escape sequence '\S'
Traceback (most recent call last):
  File "<frozen runpy>", line 198, in _run_module_as_main
  File "<frozen runpy>", line 88, in _run_code
  File "e:\Haidar\bangkit\capstone\ML-TuneHive\model-dev\.venv\Scripts\pipreqs.exe\__main__.py", line 7, in <module>
  File "E:\Haidar\bangkit\capstone\ML-TuneHive\model-dev\.venv\Lib\site-packages\pipreqs\pipreqs.py", line 609, in main
    init(args)
  File "E:\Haidar\bangkit\capstone\ML-TuneHive\model-dev\.venv\Lib\site-packages\pipreqs\pipreqs.py", line 533, in init
    candidates = get_all_imports(
                 ^^^^^^^^^^^^^^^^
  File "E:\Haidar\bangkit\capstone\ML-TuneHive\model-dev\.venv\Lib\site-packages\pipreqs\pipreqs.py", line 136, in get_all_imports
    content